In [19]:
# PASO 1: Instalar y cargar librerías
!pip install cfgrib

import xarray as xr
import numpy as np
import pandas as pd
import os


In [20]:
#PASO 2: Función para limpiar columnas comunes
def limpiar_columnas(df):
    columnas_problema = ['step', 'number', 'valid_time']
    return df.drop(columns=[col for col in columnas_problema if col in df.columns])



In [21]:
# PASO 3: Función para procesar cada archiv
def procesar_grib_anual(grib_path, anio):
    try:
        # 📁 Cargar archivo GRIB
        ds = xr.open_dataset(grib_path, engine="cfgrib")

        # 📊 Extraer variables
        u = ds['u']
        v = ds['v']
        t = ds['t']
        z = ds['z']

        # 💨 Calcular magnitud del viento y altura desde geopotencial
        speed = np.sqrt(u**2 + v**2)
        speed.name = "wind_speed"
        z_altura = z / 9.81
        z_altura.name = "altura"

        # 📈 Calcular promedios espaciales por hora y nivel
        df_u = limpiar_columnas(u.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())
        df_v = limpiar_columnas(v.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())
        df_s = limpiar_columnas(speed.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())
        df_t = limpiar_columnas(t.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())
        df_z = limpiar_columnas(z.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())
        df_a = limpiar_columnas(z_altura.mean(dim=["latitude", "longitude"]).to_dataframe().reset_index())

        # 📝 Renombrar columnas
        df_u = df_u.rename(columns={"u": "u_wind"})
        df_v = df_v.rename(columns={"v": "v_wind"})
        df_s = df_s.rename(columns={"wind_speed": "speed"})
        df_t = df_t.rename(columns={"t": "temperatura"})
        df_z = df_z.rename(columns={"z": "geopotencial"})
        df_a = df_a.rename(columns={"altura": "altura_m"})

        # 🔗 Unir todas las variables
        df = df_u.merge(df_v, on=["time", "isobaricInhPa"])
        df = df.merge(df_s, on=["time", "isobaricInhPa"])
        df = df.merge(df_t, on=["time", "isobaricInhPa"])
        df = df.merge(df_z, on=["time", "isobaricInhPa"])
        df = df.merge(df_a, on=["time", "isobaricInhPa"])
        df["año"] = anio  # Agregar columna de año

        print(f"✅ Procesado: {grib_path}")
        return df

    except Exception as e:
        print(f"❌ Error al procesar {anio}: {e}")
        return pd.DataFrame()  # Devolver vacío si falla


In [22]:
# PASO 4: Leer todos los archivos .grib
data_total = []

for anio in range(2015, 2025):
    archivo = f"sample_data/viento_{anio}.grib"
    df_anual = procesar_grib_anual(archivo, anio)
    if not df_anual.empty:
        data_total.append(df_anual)


/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Procesado: sample_data/viento_2015.grib


/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Procesado: sample_data/viento_2016.grib


/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


...

✅ Procesado: sample_data/viento_2022.grib


/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Procesado: sample_data/viento_2023.grib


/usr/local/lib/python3.11/dist-packages/cfgrib/xarray_plugin.py:131: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  vars, attrs, coord_names = xr.conventions.decode_cf_variables(


✅ Procesado: sample_data/viento_2024.grib


In [23]:
# PASO 5: Unir todo y exportar a CSV
df_total = pd.concat(data_total, ignore_index=True)
df_total.to_csv("clima_total_2015_2024.csv", index=False)
print("✅ Archivo final guardado como clima_total_2015_2024.csv")


✅ Archivo final guardado como clima_total_2015_2024.csv
